After playing around with this for awhile, there are a number of things I will need to do with this data before I can actually train a machine learning model on this.

in the following, I'm faced with two challenges
1. datetime won't convert nicely to a number (is there some other solution?)
2. Root Mean Squared Logistic Error doesn't work with negative values (how do I get around this?)

As I'm reading machine learning, consider coming back to fix these things. July 23

July 24 My solution to this was first to replace all spaces with ```_```'s but that didn't work, it turned out that ```:``` and ```-``` gave me trouble too so I replaced all these with ```_```. Then to use the Root Mean Squared Logarithmic Error we needed to normalize values so I brought in the sklearn preprocessing MinMaxScaler. 

later: Actually, I was wrong! The normalization wasn't needed for the log. THe problem was that since I was using linear regression, I was predicting negative values! This made no sense in the context of the problem anyway so I just made any negatives equal 0. I also rounded my counds to integer values. 

Finally: submitted for a score of about 1.4 - leaderboard has people at 0.33 so I'm quite a ways off, but not bad for a first try! :)


In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split




bikes = pd.read_csv('train.csv')
bikes['datetime'] = bikes['datetime'].str.replace(' ', '_', regex=False) #why does regex = false make this work?
bikes['datetime'] = bikes['datetime'].str.replace(':', '_', regex=False)
bikes['datetime'] = bikes['datetime'].str.replace('-', '_', regex=False)
#replace characters in datetime preventing us from converting to floats

test_data = pd.read_csv('test.csv')
temp_datetime = pd.read_csv('test.csv')['datetime'] #this is just to save the datetimes I need for when I build my submission file
test_data['datetime'] = test_data['datetime'].str.replace(' ', '_', regex = False) #why does regex = False make this work?
test_data['datetime'] = test_data['datetime'].str.replace(':', '_', regex = False)
test_data['datetime'] = test_data['datetime'].str.replace('-', '_', regex = False)
test_data.head()
#replace characters in datetime preventing us from converting to floats

feature_cols = ['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed']

In [2]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(bikes[feature_cols])
bikes[feature_cols]=scaler.transform(bikes[feature_cols])


In [3]:
print(bikes)
print(temp_datetime)

           datetime  season  holiday  workingday  weather      temp     atemp  \
0      0.000000e+00     0.0      0.0         0.0      0.0  0.224490  0.305068   
1      8.994239e-07     0.0      0.0         0.0      0.0  0.204082  0.288064   
2      1.798848e-06     0.0      0.0         0.0      0.0  0.204082  0.288064   
3      2.698271e-06     0.0      0.0         0.0      0.0  0.224490  0.305068   
4      3.597695e-06     0.0      0.0         0.0      0.0  0.224490  0.305068   
...             ...     ...      ...         ...      ...       ...       ...   
10881  9.999964e-01     1.0      0.0         1.0      0.0  0.367347  0.423649   
10882  9.999973e-01     1.0      0.0         1.0      0.0  0.346939  0.372860   
10883  9.999982e-01     1.0      0.0         1.0      0.0  0.326531  0.338964   
10884  9.999991e-01     1.0      0.0         1.0      0.0  0.326531  0.372860   
10885  1.000000e+00     1.0      0.0         1.0      0.0  0.306122  0.355856   

       humidity  windspeed 

In [4]:
def bikes_model(feature_cols, bikes):
    X=bikes[feature_cols]
    y=bikes['count']
    #X_test = test_data[feature_cols]
    #y_test = test_data['count']
    #setting our overall data X, and y
    #note in this example we're attempting to find an association
    #between the temperature of the day and the number of bike rentals

    X_train, X_test, y_train, y_test = train_test_split(X, y)
    #X_train[feature_cols] = scaler.transform(X_train[feature_cols])
    #X_test[feature_cols] = scaler.transform(X_test[feature_cols])
    #X_train and y_train will be used to train the model
    #X_test and y_test will be used to test the model
    #remember that all four of these variables are just subsets of the overall X and Y

    linreg = LinearRegression()
    #instantiate the model

    linreg.fit(X_train, y_train)
    # fit the model to our training set

    y_pred = linreg.predict(X_test)
    #predict our testing set
    #y_test = y_test.abs()
    
    #the log error doesn't take negative values, linear regression however can predict negative counts.
    #since this makes no sense, I make any negative predictions = 0.
    y_pred[y_pred <0] = 0
    
    error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
    #error = np.sqrt(metrics.mean_squared_log_error(y_test, y_pred))
    # calculate our metric
    
    print('RMSLE is ' + str(error))
    
    return error, y_pred, linreg

 

In [5]:
bikes.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

In [29]:
(error, sample_prediction, linreg) = bikes_model(feature_cols, bikes)

RMSLE is 1.370558424354406


In [30]:
#here we apply our model to the test set for the kaggle competition, but first we need to scale/normalize the 
#test data
scaler.fit(test_data[feature_cols])
test_data[feature_cols]=scaler.transform(test_data[feature_cols])

In [31]:
predictions = linreg.predict(test_data)
#ensure all negative values predict 0
predictions[predictions <0] = 0 

#ensure all counts are rounded
predictions = np.rint(predictions)
submission = pd.DataFrame({'datetime': temp_datetime, 'count': predictions})

In [32]:
submission.head()

,datetime,count
0,2011-01-20 00:00:00,81.0
1,2011-01-20 01:00:00,68.0
2,2011-01-20 02:00:00,68.0
3,2011-01-20 03:00:00,74.0
4,2011-01-20 04:00:00,74.0


In [33]:
submission.to_csv('linReg_submission.csv', index=False, )